## INPUT-OUTPUT ANALYSIS: THE ROLE OF THE PULP AND PAPER SECTOR IN THE BRAZILIAN ECONOMY

### Import libraries

In [1]:
import numpy as np
import pandas as pd

### Read matrices

In [2]:
# Set the year
nAnoMip = 2018
# Set the number of industries
nSetores = 68
# Set the number of products
nProdutos = 128

nColunaOferta = 7

# Define which columns the margins are in
nColunaMComercio = 1
nColunaMTransporte = 2
nColunaII = 3 
nColunaIPI = 4 
nColunaICMS = 5 
nColunaOI = 6

In [3]:
def LerMatriz(sArquivo, sNomePasta, nLinhaInicial, nColunaInicial, nLinhas, nColunas):
    dSheet = pd.read_excel(sArquivo, sNomePasta, header=None, index_col=None)
    mMatriz = dSheet.values[nLinhaInicial:nLinhaInicial+nLinhas, nColunaInicial:nColunaInicial+nColunas]
    return mMatriz

sArquivoTabelaRecursos = str(nSetores)+'_tab1_'+str(nAnoMip)+'.xls'
sArquivoTabelaUsos = str(nSetores)+'_tab2_'+str(nAnoMip)+'.xls'

mProducao = LerMatriz(sArquivoTabelaRecursos, 'producao', 5, 2, nProdutos, nSetores)
mOferta = LerMatriz(sArquivoTabelaRecursos, 'oferta', 5, 2, nProdutos, nColunaOferta)
mImportacao = LerMatriz(sArquivoTabelaRecursos, 'importacao', 5, 2, nProdutos, 1)

mCI = LerMatriz(sArquivoTabelaUsos, 'CI', 5, 2, nProdutos, nSetores)
mDemanda = LerMatriz(sArquivoTabelaUsos, 'demanda', 5, 2, nProdutos, 6)
mVA = LerMatriz(sArquivoTabelaUsos, 'VA', 5, 1, 14, nSetores)

### Estimate the Use Table at Basic Prices

#### Remove Inventories column

In [4]:
mDemandaFinalSemEstoque = np.copy(mDemanda)
mDemandaFinalSemEstoque[:,5] = 0

#### Concatenate Intermediate Consumption and Final Demand

In [5]:
mConsumoTotalSemEstoque = np.concatenate((mCI, mDemandaFinalSemEstoque), axis=1)

#### Estimate the distribution matrix of Total Consumption

In [6]:
def MatrizDistribuicao(mMatrizEntrada):
    vTotalProduto = np.sum(mMatrizEntrada, axis=1, keepdims=True)
    mMatrizSaida = mMatrizEntrada / vTotalProduto
    return mMatrizSaida

mMatrizDistribuicao = MatrizDistribuicao(mConsumoTotalSemEstoque)

#### Multiplying by margins, taxes and imports

##### Trade margins

In [7]:
# First we take the proportion of the margins in the column
vMargemC = mOferta[:,nColunaMComercio]
vMargemC = vMargemC.reshape((nProdutos, 1))
vPropMargemC = vMargemC[92:94] / np.sum(vMargemC[92:94])

# Then, we calculate the distribution matrix with the full values
mMatrizSaidaC = mMatrizDistribuicao * vMargemC

# Next, we calculate the row sums of the distribution matrix excluding the negative values
mMargemDistribuidaC = np.sum(mMatrizSaidaC[:92], axis=0) + np.sum(mMatrizSaidaC[94:], axis=0)

# And then we define the row values as the proportions
mMatrizSaidaC[92,:] = mMargemDistribuidaC * vPropMargemC[0,None] * (-1)
mMatrizSaidaC[93,:] = mMargemDistribuidaC * vPropMargemC[1,None] * (-1)

##### Transport margins

In [8]:
# First, we take the margin proportions from the column
vMargemT = mOferta[:,nColunaMTransporte]
vMargemT = vMargemT.reshape((nProdutos, 1))
vPropMargemT = vMargemT[94:98] / np.sum(vMargemT[94:98])

# next, we calculate the distribution matrix with full values
mMatrizSaidaT = mMatrizDistribuicao * vMargemT

# And then, we sum de row values of the distribution matrix, excluding the negative margins
mMargemDistribuidaT = np.sum(mMatrizSaidaT[:94], axis=0) + np.sum(mMatrizSaidaT[98:], axis=0)

# Then we define the vector's row values as proportions
mMatrizSaidaT[94,:] = mMargemDistribuidaT * vPropMargemT[0,None] * (-1)
mMatrizSaidaT[95,:] = mMargemDistribuidaT * vPropMargemT[1,None] * (-1)
mMatrizSaidaT[96,:] = mMargemDistribuidaT * vPropMargemT[2,None] * (-1)
mMatrizSaidaT[97,:] = mMargemDistribuidaT * vPropMargemT[3,None] * (-1)

##### Other margins

In [9]:
mMargemIPI = mMatrizDistribuicao * mOferta[:, nColunaIPI].reshape((nProdutos, 1))
mMargemICMS = mMatrizDistribuicao * mOferta[:, nColunaICMS].reshape((nProdutos, 1))
mMargemOI = mMatrizDistribuicao * mOferta[:, nColunaOI].reshape((nProdutos, 1))

#### Remove exportation

In [10]:
# To calculate the import margins and the imports themselves, we remove the inventories
mConsumoTotalSemExportacao = np.copy(mConsumoTotalSemEstoque)
mConsumoTotalSemExportacao[:,68] = 0

#### Distribution matrix without exports and inventories

In [11]:
mDistribuicaoSemExportacao = MatrizDistribuicao(mConsumoTotalSemExportacao)

##### Import margins and Imports

In [12]:
mMargemII = mDistribuicaoSemExportacao * mOferta[:, nColunaII].reshape((nProdutos, 1))
mMargemImportacao = mDistribuicaoSemExportacao * mImportacao[:, 0].reshape((nProdutos, 1))

### Global supply at market prices

In [13]:
mConsumoTotal = np.concatenate((mCI, mDemanda), axis=1)

### National Supply at basic prices

In [14]:
mNacionalBasico = mConsumoTotal - mMatrizSaidaC - mMargemICMS - mMargemII - mMargemImportacao - mMargemIPI - mMargemOI - mMatrizSaidaT

### IOM estimation

In [15]:
mU = mNacionalBasico[:,0:nSetores]
mE = mNacionalBasico[:,nSetores:]

mX = mVA[12,:]
mXChapeu = np.diagflat(1/mX)
mB = np.dot(mU, mXChapeu)

mProducaoV = np.copy(mProducao)
mV = mProducao.T

vQ = np.sum(mV, axis=0)
mQChapeu = np.diagflat(1/vQ)

mD = np.dot(mV, mQChapeu)
mA = np.dot(mD, mB).astype('float')
mY = np.dot(mD, mE).astype('float')
# Construção do CI setor x setor
mZ = np.dot(mD, mU).astype('float')
mI = np.eye(nSetores)
mLeontief = np.linalg.inv(mI - mA)

### IOM Construction

In [16]:
# Total Intermediate Consumption row
vZTotalO = np.sum(mZ, axis=0, keepdims=True)

# Total Import row
vMTotal = np.sum(mMargemImportacao, axis=0, keepdims=True)
vMCI = vMTotal[:, :nSetores]
vMD = vMTotal[:, nSetores:]

# Total taxes row
mTtotal = mMargemICMS + mMargemII + mMargemIPI + mMargemOI
vTtotal = np.sum(mTtotal, axis=0, keepdims=True)
vTCI = vTtotal[:, :nSetores]
vTD = vTtotal[:, nSetores:]

In [17]:
# Concatenate
mOfertaCI = np.concatenate((mZ, vZTotalO, vMCI, vTCI, mVA), axis=0)

# Sum column
vZTotalD = np.sum(mOfertaCI, axis=1, keepdims=True)

# Sum row
vTotalLinhaDN = np.sum(mY, axis=0, keepdims=True)

# Concatenate
mDemandaFinal = np.concatenate((mY, vTotalLinhaDN, vMD, vTD), axis=0)

# Concatenate
mDemanda = np.concatenate((mDemandaFinal, np.zeros([14,6], dtype=float)), axis=0)

In [18]:
vTotalDemandaFinal = np.sum(mDemanda, axis=1, keepdims=True)

vDemandaTotal = vZTotalD + vTotalDemandaFinal

mMIP = np.concatenate((mOfertaCI, vZTotalD, mDemanda, vTotalDemandaFinal, vDemandaTotal), axis=1)

#### Equilibrium Check

##### Total Supply - Total Demand

In [19]:
np.sum(np.sum(mZ, axis=1) + np.sum(mY, axis=1)) - np.sum(vZTotalO + vMCI + vTCI + mVA[0])

0.0

In [20]:
np.sum(np.sum(mZ, axis=1) + np.sum(mY, axis=1)) - np.sum(mX)

0.0

### Closed Model

In [21]:
vConsumoFamilias = mY[:,3].reshape((nSetores,1))

vAux = (mVA[1,:] + mVA[8,:]).reshape((1,nSetores))

In [22]:
vDifEOB = ((mVA[9,:] / np.sum(mVA[9,:])) * (np.sum(vConsumoFamilias) - np.sum(vAux))).reshape((1, nSetores))

In [23]:
vRendaFamilias = vAux + vDifEOB

#### Equilibrium Check

In [24]:
np.sum(vRendaFamilias) - np.sum(vConsumoFamilias)

# Result is very close to zero

-9.313225746154785e-10

#### Closed Model construction

In [25]:
mZbarra = np.concatenate((mZ, vRendaFamilias), axis=0)

nAux = np.sum(vConsumoFamilias).reshape((1,1))
vAux = np.concatenate((vConsumoFamilias, nAux), axis=0)

mZbarra = np.concatenate((mZbarra, vAux), axis=1)

vVBP = mVA[12].reshape((1,nSetores))
nAuxvbp = np.array(np.sum(vVBP)).reshape((1,1))
vVBPbarra = np.concatenate((vVBP, nAuxvbp), axis=1)

mAbarra = np.zeros([nSetores+1, nSetores+1])
mAbarra[:,:] = mZbarra[:,:] / vVBPbarra[0,:]
mIbarra = np.eye(nSetores+1)
mLeontiefbarra = np.linalg.inv(mIbarra - mAbarra)
mLeontiefbarra.shape

(69, 69)

#### Export matrices

In [26]:
writer1 = pd.ExcelWriter('Matrizes_68_2018.xlsx')

pd.DataFrame(mMIP).to_excel(writer1, sheet_name='MIP'+str(nAnoMip))
pd.DataFrame(mLeontief).to_excel(writer1, sheet_name='Leontief'+str(nAnoMip))
pd.DataFrame(mA).to_excel(writer1, sheet_name='Matriz A'+str(nAnoMip))
pd.DataFrame(mLeontiefbarra).to_excel(writer1, sheet_name='Leontief barra'+str(nAnoMip))

writer1.save()

### Multipliers

#### Sector names

In [27]:
vSetores = pd.read_excel('68_tab1_2018.xls', 'producao', header=None, index_col=None)
vSetores = vSetores.values[3,2:70]
for i in range(68):
    vSetores[i] = vSetores[i][5:]
vSetores

array(['Agricultura, inclusive o apoio à agricultura e a pós-colheita',
       'Pecuária, inclusive o apoio à pecuária',
       'Produção florestal; pesca e aquicultura',
       'Extração de carvão mineral e de minerais não-metálicos',
       'Extração de petróleo e gás, inclusive as atividades de apoio',
       'Extração de minério de ferro, inclusive beneficiamentos e a aglomeração',
       'Extração de minerais metálicos não-ferrosos, inclusive beneficiamentos',
       'Abate e produtos de carne, inclusive os produtos do laticínio e da pesca',
       'Fabricação e refino de açúcar', 'Outros produtos alimentares',
       'Fabricação de bebidas', 'Fabricação de produtos do fumo',
       'Fabricação de produtos têxteis',
       'Confecção de artefatos do vestuário e acessórios',
       'Fabricação de calçados e de artefatos de couro',
       'Fabricação de produtos da madeira',
       'Fabricação de celulose, papel e produtos de papel',
       'Impressão e reprodução de gravações',
   

##### Production

In [28]:
vMultSimplesProd = np.sum(mLeontief, axis=0, keepdims=True)
vMultTotalProd = np.sum(mLeontiefbarra, axis=0, keepdims=True)
vEfInduzido = vMultTotalProd[0,:nSetores] - vMultSimplesProd
vEfDireto = np.sum(mA, axis=0, keepdims=True)
vEfIndireto = vMultSimplesProd - vEfDireto

In [29]:
mMultiplicadores = {
    'Setores': [],
    'Total': [],
    'Simples': [],
    'Induzido': [],
    'Direto': [],
    'Indireto': []
}

for i in range(nSetores):
    mMultiplicadores['Setores'].append(vSetores[i])
    mMultiplicadores['Total'].append(vMultTotalProd[:,i][0])
    mMultiplicadores['Simples'].append(vMultSimplesProd[:,i][0])
    mMultiplicadores['Induzido'].append(vEfInduzido[:,i][0])
    mMultiplicadores['Direto'].append(vEfDireto[:,i][0])
    mMultiplicadores['Indireto'].append(vEfIndireto[:,i][0])

In [30]:
# Here, IOM was used to facilitate the operations

def FuncaoCalculoMultiplicadores(nomeoperacao, linhaoperacao, linhavbp, nsetores):
    vRequisito = mMIP[linhaoperacao,:][0:nSetores] / mMIP[linhavbp,:][0:nSetores]
    mRequisito = np.diagflat(vRequisito)

    mGerador = np.dot(mRequisito, mLeontief)
    vMultSimples = np.sum(mGerador, axis=0, keepdims=True)

    mGeradorFechado = np.dot(mRequisito, mLeontiefbarra[0:nSetores,0:nSetores])
    vMultTotal = np.sum(mGeradorFechado, axis=0, keepdims=True)

    vEfInduzido = vMultTotal - vMultSimples
    vEfeitoIndireto = vMultSimples - vRequisito

    vTipo1 = vMultSimples / vRequisito
    vTipo2 = vMultTotal / vRequisito
    
    mTabelaMultiplicadores = {
    'Setores': [],
    'Total': [],
    'Simples': [],
    'Induzido': [],
    'Direto': [],
    'Indireto': [],
    'Multiplicador Tipo 1': [],
    'Multiplicador Tipo 2': []
    }
    
    for i in range(nSetores):
        mTabelaMultiplicadores['Setores'].append(vSetores[i])
        mTabelaMultiplicadores['Total'].append(vMultTotal[:,i][0])
        mTabelaMultiplicadores['Simples'].append(vMultSimples[:,i][0])
        mTabelaMultiplicadores['Induzido'].append(vEfInduzido[:,i][0])
        mTabelaMultiplicadores['Direto'].append(vRequisito[i])
        mTabelaMultiplicadores['Indireto'].append(vEfeitoIndireto[:,i][0])
        mTabelaMultiplicadores['Multiplicador Tipo 1'].append(vTipo1[:,i][0])
        mTabelaMultiplicadores['Multiplicador Tipo 2'].append(vTipo2[:,i][0])        

    return mTabelaMultiplicadores

##### Employment

In [31]:
TabelaMultiplicadoresEmprego = FuncaoCalculoMultiplicadores('Emprego', 84, 83, 68)

##### Income

In [32]:
TabelaMultiplicadoresRenda = FuncaoCalculoMultiplicadores('Renda', 72, 83, 68)

##### GDP

In [33]:
TabelaMultiplicadoresVA = FuncaoCalculoMultiplicadores('VA', 71, 83, 68)

##### Tax

In [34]:
vRequisito = mMIP[70,:][0:nSetores] / mMIP[83,:][0:nSetores]
mRequisito = np.diagflat(vRequisito)

mGerador = np.dot(mRequisito, mLeontief)
vMultSimples = np.sum(mGerador, axis=0, keepdims=True)

mGeradorFechado = np.dot(mRequisito, mLeontiefbarra[0:nSetores,0:nSetores])
vMultTotal = np.sum(mGeradorFechado, axis=0, keepdims=True)

vEfInduzido = vMultTotal - vMultSimples
vEfeitoIndireto = vMultSimples - vRequisito

TabelaMultiplicadoresTributos = {
  'Setores': [],
  'Total': [],
  'Simples': [],
  'Induzido': [],
  'Direto': [],
  'Indireto': [],
  }
    
for i in range(nSetores):
    TabelaMultiplicadoresTributos['Setores'].append(vSetores[i])
    TabelaMultiplicadoresTributos['Total'].append(vMultTotal[:,i][0])
    TabelaMultiplicadoresTributos['Simples'].append(vMultSimples[:,i][0])
    TabelaMultiplicadoresTributos['Induzido'].append(vEfInduzido[:,i][0])
    TabelaMultiplicadoresTributos['Direto'].append(vRequisito[i])
    TabelaMultiplicadoresTributos['Indireto'].append(vEfeitoIndireto[:,i][0])      


### Linkages

##### Rows and columns means

In [35]:
# axis=0: média da coluna. axis=1: média da linha
vMediacolunasB = np.mean(mLeontief, axis=0)
vMedialinhasB = np.mean(mLeontief, axis=1)
nMediatodosB = np.mean(mLeontief)

##### Normalization

In [36]:
# Uj
vIndiceTras = vMediacolunasB / nMediatodosB
# Ui
vIndiceFrente = vMedialinhasB / nMediatodosB

##### Coefficient of Variation

In [37]:
# The smaller the coefficient, the more uniform its impact
vVJ = np.std(mLeontief, axis=0, ddof=1) / vMediacolunasB
vVI = np.std(mLeontief, axis=1, ddof=1) / vMedialinhasB

### Price Model

In [38]:
d = mMIP[69:72,:nSetores]
d = d / mMIP[83,:nSetores]

In [39]:
p = np.dot(mLeontief.T, d.T)
p.shape

(68, 3)

#### Checking

In [40]:
np.sum(p, axis=1)

# Very close to 1

array([1.0000000000000002, 1.0000000000000004, 0.9999999999999997,
       0.9999999999999999, 1.0, 0.9999999999999998, 0.9999999999999998,
       1.0000000000000002, 0.9999999999999998, 0.9999999999999998,
       0.9999999999999998, 1.0000000000000002, 1.0000000000000004,
       0.9999999999999998, 1.0000000000000002, 1.0000000000000004,
       1.0000000000000002, 0.9999999999999999, 1.0, 0.9999999999999999,
       0.9999999999999994, 0.9999999999999998, 0.9999999999999999, 1.0,
       1.0000000000000002, 1.0000000000000002, 1.0000000000000002,
       1.0000000000000004, 0.9999999999999997, 0.9999999999999996,
       1.0000000000000002, 1.0, 1.0, 0.9999999999999999,
       0.9999999999999998, 0.9999999999999998, 0.9999999999999998,
       0.9999999999999999, 0.9999999999999999, 1.0, 1.0000000000000002,
       1.0, 1.0, 1.0000000000000002, 0.9999999999999997, 1.0, 1.0, 1.0,
       0.9999999999999998, 1.0000000000000004, 1.0, 1.0000000000000004,
       0.9999999999999998, 0.9999999999999

### Pure Linkages

In [41]:
mIPL = np.zeros([nSetores, 3])
mYtotal = np.sum(mY, axis=1, keepdims=True)

In [42]:
for n in range(nSetores):
    i = n
    j = i
    vRestoEconomia = [True] * nSetores
    vRestoEconomia[i] = False
    
    Yj = mYtotal[i]
    Yr = mYtotal[vRestoEconomia]
    Ajj = mA[i, j]
    Deltaj = (1/(1 - Ajj))
    Ajr = mA[i, vRestoEconomia].reshape((1,nSetores-1))
    Arj = mA[vRestoEconomia, i].reshape((nSetores-1,1))
    Arr = mA[np.ix_(vRestoEconomia, vRestoEconomia)]
    mI = np.eye(nSetores - 1)
    Deltar = np.linalg.inv(mI - Arr)
    
    PFL = np.dot(Deltaj,Ajr).dot(Deltar).dot(Yr)
    PBL = np.sum(Deltar.dot(Arj).dot(Deltaj).dot(Yj))
    PTL = PBL + PFL
    
    mIPL[n,0] = PBL
    mIPL[n,1] = PFL
    mIPL[n,2] = PTL
    
mIPLn = (np.sum(mIPL, axis=0) / nSetores)
mIPLn = np.diagflat(1 / mIPLn)
mIPLN = np.dot(mIPL, mIPLn) 

### Results

##### Backward and Forward Linkages

In [43]:
df_ind_tras_frente = {
    'Setores': vSetores,
    'BL': vMediacolunasB,
    'PD': vIndiceTras,
    'FL': vMedialinhasB,
    'SD': vIndiceFrente
}

In [44]:
df_ind_tras_frente = pd.DataFrame(df_ind_tras_frente)

In [45]:
df_ind_tras_frente['Ranking BL'] = df_ind_tras_frente['PD'].rank(ascending=False)
df_ind_tras_frente['Ranking FL'] = df_ind_tras_frente['SD'].rank(ascending=False)

In [46]:
df_ind_tras_frente = df_ind_tras_frente[['Setores', 'BL', 'PD', 'Ranking BL', 'FL', 'SD', 'Ranking FL']]

df_ind_tras_frente[df_ind_tras_frente['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,BL,PD,Ranking BL,FL,SD,Ranking FL
0,"Fabricação de celulose, papel e produtos de papel",0.030472,1.140986,13.0,0.028959,1.084361,17.0


##### Power and sensitivity of dispersion

In [47]:
df_ind_pd_sd = {
    'Setores': vSetores,
    'CV PD': vVJ,
    'CV SD': vVI
}

df_ind_pd_sd = pd.DataFrame(df_ind_pd_sd)

df_ind_pd_sd['Ranking PD'] = df_ind_pd_sd['CV PD'].rank(ascending=False)
df_ind_pd_sd['Ranking SD'] = df_ind_pd_sd['CV SD'].rank(ascending=False)

df_ind_pd_sd = df_ind_pd_sd[['Setores', 'CV PD', 'Ranking PD', 'CV SD', 'Ranking SD']]
df_ind_pd_sd[df_ind_pd_sd['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,CV PD,Ranking PD,CV SD,Ranking SD
0,"Fabricação de celulose, papel e produtos de papel",4.754894,40.0,4.977179,48.0


##### Key sectors

In [48]:
setores_chave = {
    'Setores': [],
    'BL': [],
    'PD': [],
    'FL': [],
    'SD': []
}

In [49]:
for i in range(68):
    if (vIndiceTras[i] > 1) & (vIndiceFrente[i] > 1):
        setores_chave['Setores'].append(vSetores[i])
        setores_chave['BL'].append(vMediacolunasB[i])
        setores_chave['PD'].append(vIndiceTras[i])
        setores_chave['FL'].append(vMedialinhasB[i])
        setores_chave['SD'].append(vIndiceFrente[i])

In [50]:
setores_chave = pd.DataFrame(setores_chave)
setores_chave

,Setores,BL,PD,FL,SD
0,Outros produtos alimentares,0.034014,1.273646,0.026873,1.006244
1,"Fabricação de celulose, papel e produtos de papel",0.030472,1.140986,0.028959,1.084361
2,Refino de petróleo e coquerias,0.034221,1.281389,0.071247,2.667784
3,Fabricação de químicos orgânicos e inorgânicos...,0.029467,1.103388,0.050681,1.897711
4,"Fabricação de defensivos, desinfestantes, tint...",0.030344,1.136216,0.029757,1.114213
5,Fabricação de produtos de borracha e de materi...,0.030245,1.132509,0.031825,1.191654
6,"Produção de ferro-gusa/ferroligas, siderurgia ...",0.031093,1.164264,0.033250,1.245007
7,Metalurgia de metais não-ferrosos e a fundição...,0.031241,1.169781,0.027154,1.016753
8,"Fabricação de produtos de metal, exceto máquin...",0.030360,1.136807,0.028009,1.048758
9,"Manutenção, reparação e instalação de máquinas...",0.026716,1.000367,0.031860,1.192985


##### Pure Linkages

In [51]:
df_indices_puros = {
    'Setores': vSetores,
    'PBL': mIPLN[:,0],
    'PFL': mIPLN[:,1],
    'PTL': mIPLN[:,2]
}

In [52]:
df_indices_puros = pd.DataFrame(df_indices_puros)

df_indices_puros['Ranking PBL'] = df_indices_puros['PBL'].rank(ascending=False)
df_indices_puros['Ranking PFL'] = df_indices_puros['PFL'].rank(ascending=False)
df_indices_puros['Ranking PTL'] = df_indices_puros['PTL'].rank(ascending=False)

df_indices_puros = df_indices_puros[['Setores', 'PBL', 'Ranking PBL', 'PFL', 'Ranking PFL', 'PTL', 'Ranking PTL']]
df_indices_puros.sort_values(by='PTL', ascending=False)

,Setores,PBL,Ranking PBL,PFL,Ranking PFL,PTL,Ranking PTL
41,"Comércio por atacado e a varejo, exceto veícul...",5.409040,1.0,6.734391,1.0,6.070550,1.0
39,Construção,5.369328,2.0,0.881289,25.0,3.129255,2.0
52,"Intermediação financeira, seguros e previdênci...",1.436476,14.0,4.662280,2.0,3.046541,3.0
18,Refino de petróleo e coquerias,1.864889,10.0,4.045099,3.0,2.953077,4.0
7,"Abate e produtos de carne, inclusive os produt...",5.129641,3.0,0.548978,35.0,2.843338,5.0
...,...,...,...,...,...,...,...
3,Extração de carvão mineral e de minerais não-m...,0.018636,65.0,0.289973,47.0,0.154066,64.0
11,Fabricação de produtos do fumo,0.267128,48.0,0.001162,67.0,0.134379,65.0
6,"Extração de minerais metálicos não-ferrosos, i...",0.130491,58.0,0.126378,58.0,0.128438,66.0
48,Edição e edição integrada à impressão,0.103407,60.0,0.137398,57.0,0.120373,67.0


In [53]:
df_indices_puros[df_indices_puros['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,PBL,Ranking PBL,PFL,Ranking PFL,PTL,Ranking PTL
0,"Fabricação de celulose, papel e produtos de papel",0.659603,31.0,0.801274,27.0,0.730314,32.0


##### Price formation

In [54]:
df_precos = {
    'Setores': vSetores,
    '% Importação': p[:, 0],
    '% Impostos': p[:, 1],
    '% Valor Adicionado': p[:,2]
}

df_precos = pd.DataFrame(df_precos)
df_precos[df_precos['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,% Importação,% Impostos,% Valor Adicionado
0,"Fabricação de celulose, papel e produtos de papel",0.170019,0.0942676,0.735713


In [55]:
df_precos['% Importação'].mean()

0.13544402251911747

In [56]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_precos.sort_values(by='% Importação', ascending=False).reset_index(drop=True))

,Setores,% Importação,% Impostos,% Valor Adicionado
0,Fabricação de outros equipamentos de transport...,0.403829,0.0734221,0.522749
1,"Fabricação de equipamentos de informática, pro...",0.391991,0.118597,0.489412
2,Fabricação de químicos orgânicos e inorgânicos...,0.388002,0.108735,0.503263
3,"Fabricação de defensivos, desinfestantes, tint...",0.321716,0.102775,0.575509
4,Fabricação de produtos de borracha e de materi...,0.263568,0.106155,0.630278
5,Metalurgia de metais não-ferrosos e a fundição...,0.256784,0.0908236,0.652393
6,Refino de petróleo e coquerias,0.253931,0.18791,0.558159
7,"Fabricação de automóveis, caminhões e ônibus, ...",0.249646,0.118429,0.631924
8,Fabricação de máquinas e equipamentos elétricos,0.239453,0.101371,0.659176
9,"Manutenção, reparação e instalação de máquinas...",0.23825,0.083035,0.678715


In [57]:
df_precos['% Impostos'].mean()

0.07887838309877655

In [58]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_precos.sort_values(by='% Impostos', ascending=False).reset_index(drop=True))

,Setores,% Importação,% Impostos,% Valor Adicionado
0,Refino de petróleo e coquerias,0.253931,0.18791,0.558159
1,Transporte aéreo,0.226044,0.161688,0.612268
2,"Energia elétrica, gás natural e outras utilidades",0.0953234,0.131594,0.773082
3,Fabricação de bebidas,0.114439,0.119046,0.766515
4,"Fabricação de equipamentos de informática, pro...",0.391991,0.118597,0.489412
5,"Fabricação de automóveis, caminhões e ônibus, ...",0.249646,0.118429,0.631924
6,"Fabricação de produtos de limpeza, cosméticos/...",0.225273,0.114505,0.660222
7,Fabricação de produtos de minerais não-metálicos,0.157987,0.109376,0.732637
8,Fabricação de calçados e de artefatos de couro,0.135335,0.108842,0.755823
9,Fabricação de químicos orgânicos e inorgânicos...,0.388002,0.108735,0.503263


In [59]:
df_precos['% Valor Adicionado'].mean()

0.7856775943821063

In [60]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_precos.sort_values(by='% Valor Adicionado', ascending=False).reset_index(drop=True))

,Setores,% Importação,% Impostos,% Valor Adicionado
0,Serviços domésticos,0,0,1
1,Atividades imobiliárias,0.00848222,0.0112707,0.980247
2,Educação pública,0.0241696,0.0205879,0.955242
3,"Atividades de vigilância, segurança e investig...",0.0281004,0.025346,0.946554
4,"Administração pública, defesa e seguridade social",0.0341803,0.0375573,0.928262
5,"Atividades jurídicas, contábeis, consultoria e...",0.0361478,0.0361363,0.927716
6,Produção florestal; pesca e aquicultura,0.0387735,0.0360848,0.925142
7,Educação privada,0.0432919,0.0342446,0.922464
8,"Intermediação financeira, seguros e previdênci...",0.0328494,0.0449223,0.922228
9,Outras atividades administrativas e serviços c...,0.0489306,0.0476584,0.903411


##### Production Multipliers

In [61]:
mMultiplicadores = pd.DataFrame(mMultiplicadores)
mMultiplicadores.sort_values(by='Total', ascending=False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(mMultiplicadores.sort_values(by='Total', ascending=False).reset_index(drop=True))

,Setores,Total,Simples,Induzido,Direto,Indireto
0,"Abate e produtos de carne, inclusive os produt...",3.992919,2.497733,1.495186,0.776931,1.720802
1,Fabricação e refino de açúcar,3.962771,2.390283,1.572489,0.790644,1.599638
2,Serviços domésticos,3.862777,1.000000,2.862777,0.000000,1.000000
3,Educação pública,3.745857,1.225757,2.520100,0.134944,1.090813
4,Fabricação de biocombustíveis,3.718053,2.334341,1.383712,0.731969,1.602372
5,Saúde pública,3.701542,1.467727,2.233815,0.274497,1.193230
6,Outros produtos alimentares,3.663945,2.312983,1.350961,0.709291,1.603693
7,Educação privada,3.663692,1.387922,2.275771,0.238698,1.149224
8,"Atividades de vigilância, segurança e investig...",3.622674,1.252756,2.369918,0.155040,1.097715
9,"Pecuária, inclusive o apoio à pecuária",3.612762,1.885100,1.727662,0.450258,1.434842


In [62]:
mMultiplicadores[mMultiplicadores['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,Total,Simples,Induzido,Direto,Indireto
0,"Fabricação de celulose, papel e produtos de papel",3.199667,2.072068,1.127599,0.579061,1.493008


##### Employment Multipliers

In [63]:
TabelaMultiplicadoresEmprego = pd.DataFrame(TabelaMultiplicadoresEmprego)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(TabelaMultiplicadoresEmprego.sort_values(by='Multiplicador Tipo 2', ascending=False).reset_index(drop=True))

,Setores,Total,Simples,Induzido,Direto,Indireto,Multiplicador Tipo 1,Multiplicador Tipo 2
0,Refino de petróleo e coquerias,6.875053,4.709924,2.165129,0.048538,4.661386,97.034937,141.641410
1,"Extração de petróleo e gás, inclusive as ativi...",6.763020,4.239479,2.523541,0.221937,4.017542,19.102172,30.472701
2,"Extração de minério de ferro, inclusive benefi...",7.499620,5.057648,2.441972,0.383857,4.673790,13.175857,19.537525
3,Fabricação de produtos do fumo,17.358557,13.132735,4.225823,1.133172,11.999563,11.589358,15.318556
4,Fabricação de químicos orgânicos e inorgânicos...,7.565341,5.057071,2.508270,0.514105,4.542965,9.836643,14.715544
5,"Fabricação de automóveis, caminhões e ônibus, ...",12.295891,8.390783,3.905108,0.854764,7.536019,9.816490,14.385128
6,"Energia elétrica, gás natural e outras utilidades",6.144196,3.785119,2.359077,0.490185,3.294934,7.721812,12.534435
7,"Produção de ferro-gusa/ferroligas, siderurgia ...",9.649447,6.548205,3.101242,0.818379,5.729826,8.001436,11.790931
8,"Abate e produtos de carne, inclusive os produt...",28.541748,23.740458,4.801290,2.616916,21.123542,9.071921,10.906634
9,Fabricação de biocombustíveis,19.018273,14.574944,4.443329,1.798323,12.776620,8.104740,10.575557


In [64]:
TabelaMultiplicadoresEmprego[TabelaMultiplicadoresEmprego['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,Total,Simples,Induzido,Direto,Indireto,Multiplicador Tipo 1,Multiplicador Tipo 2
0,"Fabricação de celulose, papel e produtos de papel",12.029134,8.408228,3.620907,1.841115,6.567113,4.566921,6.533614


##### GDP Multipliers

In [65]:
TabelaMultiplicadoresVA = pd.DataFrame(TabelaMultiplicadoresVA)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(TabelaMultiplicadoresVA.sort_values(by='Multiplicador Tipo 2', ascending=False).reset_index(drop=True))

,Setores,Total,Simples,Induzido,Direto,Indireto,Multiplicador Tipo 1,Multiplicador Tipo 2
0,Refino de petróleo e coquerias,0.674958,0.558159,0.116799,0.094605,0.463553,5.899862,7.134455
1,"Fabricação de automóveis, caminhões e ônibus, ...",0.842588,0.631924,0.210663,0.123529,0.508395,5.115585,6.820957
2,"Abate e produtos de carne, inclusive os produt...",1.046759,0.787751,0.259008,0.156738,0.631013,5.025923,6.678420
3,Fabricação e refino de açúcar,1.057296,0.784897,0.272399,0.163398,0.621499,4.803581,6.470669
4,Outros produtos alimentares,1.000066,0.766041,0.234025,0.203199,0.562843,3.769913,4.921616
5,Fabricação de biocombustíveis,1.022287,0.782589,0.239698,0.210919,0.571670,3.710371,4.846815
6,Fabricação de produtos do fumo,1.011364,0.783400,0.227964,0.228240,0.555159,3.432348,4.431141
7,"Fabricação de defensivos, desinfestantes, tint...",0.749303,0.575509,0.173795,0.172714,0.402795,3.332154,4.338412
8,Fabricação de químicos orgânicos e inorgânicos...,0.638573,0.503263,0.135310,0.148719,0.354544,3.383980,4.293815
9,"Fabricação de equipamentos de informática, pro...",0.640806,0.489412,0.151393,0.150661,0.338752,3.248444,4.253306


In [66]:
TabelaMultiplicadoresVA[TabelaMultiplicadoresVA['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,Total,Simples,Induzido,Direto,Indireto,Multiplicador Tipo 1,Multiplicador Tipo 2
0,"Fabricação de celulose, papel e produtos de papel",0.931045,0.735713,0.195332,0.289977,0.445737,2.537146,3.210758


##### Income Multipliers

In [67]:
TabelaMultiplicadoresRenda = pd.DataFrame(TabelaMultiplicadoresRenda)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(TabelaMultiplicadoresRenda.sort_values(by='Multiplicador Tipo 2', ascending=False).reset_index(drop=True))

,Setores,Total,Simples,Induzido,Direto,Indireto,Multiplicador Tipo 1,Multiplicador Tipo 2
0,Refino de petróleo e coquerias,0.228055,0.179876,0.048180,0.019870,0.160005,9.052462,11.477162
1,"Extração de minério de ferro, inclusive benefi...",0.251797,0.197457,0.054340,0.048762,0.148695,4.049404,5.163798
2,Atividades imobiliárias,0.061248,0.038566,0.022682,0.012384,0.026182,3.114062,4.945508
3,"Abate e produtos de carne, inclusive os produt...",0.470975,0.364134,0.106841,0.102997,0.261137,3.535381,4.572700
4,"Fabricação de automóveis, caminhões e ônibus, ...",0.462167,0.375268,0.086899,0.109001,0.266267,3.442782,4.240007
5,Metalurgia de metais não-ferrosos e a fundição...,0.342077,0.275048,0.067029,0.085402,0.189646,3.220625,4.005486
6,Fabricação de químicos orgânicos e inorgânicos...,0.286119,0.230303,0.055815,0.071881,0.158423,3.203967,3.980469
7,Outros produtos alimentares,0.428041,0.331506,0.096535,0.109010,0.222496,3.041066,3.926629
8,"Produção de ferro-gusa/ferroligas, siderurgia ...",0.348693,0.279682,0.069011,0.089301,0.190381,3.131892,3.904675
9,"Energia elétrica, gás natural e outras utilidades",0.255552,0.203057,0.052495,0.066519,0.136538,3.052598,3.841774


In [68]:
TabelaMultiplicadoresRenda[TabelaMultiplicadoresRenda['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,Total,Simples,Induzido,Direto,Indireto,Multiplicador Tipo 1,Multiplicador Tipo 2
0,"Fabricação de celulose, papel e produtos de papel",0.403356,0.322782,0.080574,0.127999,0.194783,2.52176,3.151254


##### Tax Multipliers

In [69]:
TabelaMultiplicadoresTributos = pd.DataFrame(TabelaMultiplicadoresTributos)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(TabelaMultiplicadoresTributos.sort_values(by='Total', ascending=False).reset_index(drop=True))

,Setores,Total,Simples,Induzido,Direto,Indireto
0,Refino de petróleo e coquerias,0.197428,0.187910,0.009518,0.109962,0.077948
1,Transporte aéreo,0.177350,0.161688,0.015662,0.109551,0.052136
2,"Energia elétrica, gás natural e outras utilidades",0.141965,0.131594,0.010371,0.075767,0.055827
3,Fabricação de bebidas,0.136544,0.119046,0.017498,0.069144,0.049902
4,"Fabricação de automóveis, caminhões e ônibus, ...",0.135597,0.118429,0.017168,0.062427,0.056003
5,"Fabricação de produtos de limpeza, cosméticos/...",0.131523,0.114505,0.017018,0.063735,0.050770
6,"Fabricação de equipamentos de informática, pro...",0.130935,0.118597,0.012338,0.078698,0.039899
7,Fabricação de calçados e de artefatos de couro,0.129937,0.108842,0.021095,0.065086,0.043756
8,Fabricação de produtos de minerais não-metálicos,0.129362,0.109376,0.019986,0.057212,0.052163
9,"Abate e produtos de carne, inclusive os produt...",0.128554,0.107447,0.021108,0.044704,0.062743


In [70]:
TabelaMultiplicadoresTributos[TabelaMultiplicadoresTributos['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,Total,Simples,Induzido,Direto,Indireto
0,"Fabricação de celulose, papel e produtos de papel",0.110186,0.094268,0.015918,0.046514,0.047754


##### Chocks

In [100]:
vSetores[16]

'Fabricação de celulose, papel e produtos de papel'

In [101]:
mY[16,0]

33268.78930471119

In [97]:
### Increase of 5.41% in the pulp and paper sector's exports

mAux = np.zeros((nSetores,1))

nChoque = 0.0541
nDeltaY = (nChoque * mY[16,0]).reshape((1,1))

mAux[16,0] = nDeltaY

# Closed Model
mDeltaX = np.dot(mLeontiefbarra[0:nSetores, 0:nSetores], mAux)

nDeltaVBP = np.sum(mDeltaX)
nDeltaVBPperc = nDeltaVBP / np.sum(mVA[12,:])

vRelacaoVA = (mVA[0,:] / mVA[12,:]).reshape((nSetores,1))
vDeltaVA = vRelacaoVA * mDeltaX
nDeltaVA = np.sum(vDeltaVA)
nDeltaVAperc = nDeltaVA / np.sum(mVA[0])

vRelacaoRemuneracoes = (mVA[1,:] / mVA[12,:]).reshape((nSetores,1))
vDeltaRemuneracoes = vRelacaoRemuneracoes * mDeltaX
nDeltaRemuneracoes = np.sum(vDeltaRemuneracoes)
nDeltaRemuneracoesperc = nDeltaRemuneracoes / np.sum(mVA[1])

vRelacaoOcupacoes = (mVA[13,:] / mVA[12,:]).reshape((nSetores,1))
vDeltaOcupacoes = vRelacaoOcupacoes * mDeltaX
nDeltaOcupacoes= np.sum(vDeltaOcupacoes)
nDeltaOcupacoesperc = nDeltaOcupacoes / np.sum(mVA[13])

vRelacaoImpostos = (mMIP[70,:nSetores] / mVA[12,:]).reshape((nSetores,1))
vDeltaImpostos = vRelacaoImpostos * mDeltaX
nDeltaImpostos = np.sum(vDeltaImpostos)
nDeltaImpostosperc = nDeltaImpostos / np.sum(mVA[13])

In [110]:
nDeltaImpostos

198.3173014977913

In [111]:
nDeltaImpostosperc

1.9006783478171905e-06

In [106]:
nDeltaRemuneracoes

725.9767916535383

In [107]:
nDeltaRemuneracoesperc

0.00023757549780482328

In [108]:
nDeltaOcupacoes

21650.535290796997

In [109]:
nDeltaOcupacoesperc

0.0002074993121380694

In [102]:
nDeltaVBP

4440.443588949592

In [103]:
nDeltaVBPperc

0.00036972855051324615

In [104]:
nDeltaVA

1675.7337875132998

In [105]:
nDeltaVAperc

0.0002787709153012818